# Part II - KES

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import string
import math

In [29]:
ALPHABET = string.ascii_uppercase

In [30]:
enc_msg = "HSQQD XHDRP YFKWV HNHDL OULLQ DDWVW BDWWA RJULS".upper().replace(" ", "")
enc_msg

'HSQQDXHDRPYFKWVHNHDLOULLQDDWVWBDWWARJULS'

In [31]:
# phone book keys of the day
name, streetname, housenumber, phonenr = "John Smith", "Lagardsveien", 3, "51634782"

### Task 1. Present your cryptanalysis on this KES.

#### TODO

### Task 2. What is the clear-text of the message above?

In [32]:
test_message = "testing the ciphes"
test_key = "key"
len(test_message.replace(" ", ""))

16

#### Vigenere Cipher

In [33]:
def vigener_cipher(text, key, mode = 'encrypt'):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")
    key = key.upper()

    for i in range(len(text)):
        if mode == "encrypt":
            message += ALPHABET[(ord(text[i]) + ord(key[i % len(key)])) % 26]
        else:
            message += ALPHABET[(ord(text[i]) - ord(key[i % len(key)])) % 26]
    return message


In [34]:
test_enc_vigener = vigener_cipher(test_message, test_key)
test_dec_vigener = vigener_cipher(test_enc_vigener, test_key, mode = 'decrypt')
print(test_enc_vigener, test_dec_vigener)

DIQDMLQXFOGGZLCC TESTINGTHECIPHES


#### Caesar Cipher

In [35]:
def ceasar_cipher(text, key, mode="encrypt"):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")
    cipher_alpha = ALPHABET[key:] + ALPHABET[:key] # shift the alphabet by key positions

    # mapping between the two alphabets
    alpha_dict = dict(zip(cipher_alpha, ALPHABET)) if mode == "decrypt" else dict(zip(ALPHABET, cipher_alpha))
    for c in text:
        message += alpha_dict[c]
    return message

In [36]:
test_shift = 3
test_enc_ceasar = ceasar_cipher(test_message, test_shift)
test_dec_ceasar = ceasar_cipher(test_enc_ceasar, test_shift, mode="decrypt")
print(test_enc_ceasar, test_dec_ceasar)

WHVWLQJWKHFLSKHV TESTINGTHECIPHES


#### Transposition Cipher

In [37]:
def transposition_cipher(text, key, mode="encrypt"):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")

    cols, rows = len(key), math.ceil(len(text) / len(key))
    matrix = np.array([[""] * cols] * rows) # create transposition matrix
    letter_index = 0

    if mode == "encrypt":
        # fill matrix
        for i in range(len(text)):
            for j in range(cols):
                if letter_index < len(text):
                    matrix[i, j] = text[letter_index]
                    letter_index += 1
        # read matrix => encryoted message
        for k in sorted(key):
            for i in range(rows):
                message += matrix[i, key.index(k)]
    else:
        for i in key:
            col = int(i) - 1 # string col index to int
            letter = text[rows*col:(rows*col) + rows] # get letter from rows and col indexes
            # fill matrix
            for j in range(rows):
                matrix[j, letter_index] = letter[j]
            letter_index += 1
        
        # read matrix => decrupt message
        for i in range(rows):
            for j in range(cols):
                message += matrix[i, j]
                    
    return message

In [50]:
test_enc_transposition = transposition_cipher(test_message, phonenr)
test_dec_transposition = transposition_cipher(test_enc_transposition, phonenr, mode="decrypt")
print(test_enc_transposition, test_dec_transposition)

EAEAGETPTILHTSTSMXNIEGNR TESTINGASAMPLINGTEXTHERE


### Task 3. How would you improve this KES?

...

### Task 4. Implement your improved KES and encrypt the same message

In [52]:
def KES_cipher(text, street_name, house_number, phone_nr, mode="encrypt", withimprovments=False):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")

    if mode == "encrypt":
        message = transposition_cipher(text, phone_nr)
        message = ceasar_cipher(message, house_number)
        if withimprovments:
            message = vigener_cipher(message, street_name)
    else:
        if withimprovments:
            message = vigener_cipher(text, street_name, mode="decrypt")
        if withimprovments:
            message = transposition_cipher(message, phone_nr, mode="decrypt")
        else:
            message = ceasar_cipher(text, house_number, mode="decrypt")
        message = transposition_cipher(message, phone_nr, mode="decrypt")
    return message


In [54]:
test_enc_KES = KES_cipher(test_message, streetname, housenumber, phonenr)
test_dec_KES = KES_cipher(test_enc_KES, streetname, housenumber, phonenr, mode="decrypt")
print(test_enc_KES, test_dec_KES)

HHWVWLLSWKVFQKJH TESTINGTHECIPHES


##### Decrypting the first message

In [55]:
KES_cipher(enc_msg, streetname, housenumber, phonenr, mode="decrypt")

'LEAVEYOURPACKAGEINTHETRAINSTATIONATSIXPM'

In [53]:
KES_cipher('LEAVEYOURPACKAGEINTHETRAINSTATIONATSIXPM', streetname, housenumber, phonenr)

'HSQQDXHDRPYFKWVHNHDLOULLQDDWVWBDWWARJULS'

### Task 5. Webserver using improved KES

In [15]:
from flask import Flask, request
app = Flask(__name__)

@app.route('/')
def index():
    return "To encrypt a message: /encrypt?text=mymessage. <br> To decrypt a cipher: /decrypt?cipher=mycipher'"

@app.route('/encrypt')
def encrypt():
    message = request.args.get('text')
    return "Cipher " + KES_cipher(message, streetname, housenumber, phonenr, mode="encrypt", withimprovments=True)

@app.route('/decrypt')
def decrypt():
    message = request.args.get('cipher')
    return "Plaintext " + KES_cipher(message, streetname, housenumber, phonenr, mode="decrypt", withimprovments=True)

# if __name__ == '__main__':
#     app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Sep/2022 12:10:48] "GET /encrypt?text=LEAVEYOURPACKAGEINTHETRAINSTATIONATSIXPM HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 12:11:03] "GET /encrypt?text=SSWQUAZYVXCSVWBHEKVGSCPYBDJWMZTYAEEEUURS HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 12:11:20] "GET /decrypt?cipher=SSWQUAZYVXCSVWBHEKVGSCPYBDJWMZTYAEEEUURS HTTP/1.1" 200 -
